# Подключение библиотек

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Функция задания границ поля робота

In [3]:
def check_points(event,x,y,flags,param):
    global flag
    if not flag:
        if event == cv2.EVENT_MBUTTONDBLCLK:
            if len(inp) > 0:
                cv2.circle(img,inp[-1],5,(0,0,0),-1)
                inp.pop()
        if event == cv2.EVENT_LBUTTONDBLCLK:
            if len(inp) < 4:
                cv2.circle(img,(x,y),5,(0,0,255),-1)
                inp.append([x,y])
        if event == cv2.EVENT_RBUTTONDBLCLK:
            if len(inp) == 4:
                flag=True
        

# Функция обработки данных положения робота

In [4]:
def text_izm(txt):
    s = ['x: ', 'y: ', 'phi: ', 'rho: ', 'theta: ']
    c = list(map(str,txt))
    return '; '.join([i+j for i,j in zip(s,c)])

# Основная программа

In [5]:
# Создание связи видеопотока с камеры или её записи
cam = cv2.VideoCapture('WIN_20230401_13_46_34_Pro.mp4')
#cam = cv2.VideoCapture(0) #для съемки с самой камеры

#список для хранения координат границ поля перемещения робота
inp = []

#переменная для хранения слоя изображения,
# на котором обозначены точки границ поля перемещения робота
img=np.zeros((576,1024,3), np.uint8) 


col = np.uint8([0,0])
#Переменная-флаг для обозначения, что произошло преобразование Перспективы поля робота
flag=False 

#Размерные коэффициенты координат поля робота в метрах/пиксель
kx = 7.5/600
ky = 4.5/600

#Список координат робота в формате [x,y,phi,rho,theta], где
# x, y - декартовые координаты робота в метрах
# phi - угол ориентации робота
# rho, theta - полярные координаты робота
izm = []

def nothing(*arg):
    pass

cv2.namedWindow('mask0')
cv2.namedWindow('mask1')
cv2.createTrackbar('h1', 'mask0', 107, 180, nothing)
cv2.createTrackbar('h2', 'mask1', 89, 180, nothing)

# Основной цикл программы
while(cam.isOpened()):
    # считывание кадров видео по очереди, пока не закончаться
    ret, frame = cam.read()
    if frame is None:
        break
    
    # изменение размера кадра с сохранением пропорций
    w = 1024
    f = float(w)/frame.shape[1]
    new_size=(w, int(frame.shape[0]*f)) #ширина=1024, высота=576)
    frame=cv2.resize(frame, new_size, interpolation=cv2.INTER_AREA)
    
    # Если преобразование Перспективы вызвано и возможно, то выполняется код ниже
    if flag:
        
        #Преобразование Перспективы изображения
        pts1 = np.float32(inp) #Точки исходного изображения
        pts2 = np.float32([[0,0], [600,0],[600,600],[0,600]]) #Точки преобразованного изображения
        M = cv2.getPerspectiveTransform(pts1,pts2) #Матрица преобразования Перспективы
        frame2 = cv2.warpPerspective(frame,M, (600,600)) #Преобразованное изображение
        
        #Преобразование изображения из формата BGR в формат HSV
        hsv = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
        h1 = cv2.getTrackbarPos('h1', 'mask0')
        h2 = cv2.getTrackbarPos('h2', 'mask1')
        #Задание пороговых значений цвета метки №1
        col0_min = np.array([h1-10,50,50])
        col0_max = np.array([h1+10,200,200])
        #Создание маски изображения для метки №1
        mask0 = cv2.inRange(hsv, col0_min, col0_max)
        
        #Задание пороговых значений цвета метки №2
        col1_min = np.array([h2-10,50,50])
        col1_max = np.array([h2+10,200,200])
        #Создание маски изображения для метки №2
        mask1 = cv2.inRange(hsv, col1_min, col1_max)
        
        #Фильтрация масок изображения с использованием морфологической операцией Open
        kernel = np.ones((9,9),np.uint8) #Структурный элемент
        #Создание отфильтрованых масок
        op0 = cv2.morphologyEx(mask0, cv2.MORPH_OPEN, kernel)
        op1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel)
        
        
        M0 = cv2.moments(op0,1) #Вычисление моментов маски №1
        M1 = cv2.moments(op1,1) #Вычисление моментов маски №2
        if M0['m00'] == 0:
            M0['m00'] = 1
        if M1['m00'] == 0:
            M1['m00'] = 1
            
        cx0,cy0 = M0['m01']/M0['m00'], M0['m10']/M0['m00'] #Поиск центра метки №1
        cx1,cy1 = M1['m01']/M1['m00'], M1['m10']/M1['m00'] #Поиск центра метки №2
        
        #M10 - моменты вдоль горизонтальной оси (Mx, M01 - вдоль вертикальной My, M00 - сумма интенсивности всго изображения
        #Но в данном случае это моменты для системы координат, где X-направлена вниз, Y-вправо
        
        #Вычисление декартовых координат робота и угла ориентации
        cx, cy = kx*(cx0+cx1)/2, ky*(cy0+cy1)/2
        vx, vy = kx*(cx0-cx1), ky*(cy0-cy1)
        phi = np.angle(complex(vx,vy),True)
        #Вычисление полярных координат
        rho = np.sqrt(cx**2 + cy**2) #Радиус вектор
        theta = np.angle(complex(cx,cy),True) #угол радиус вектора
        
        #Добавление данных в спиксок измерений
        izm.append(text_izm([np.round(cx, 2),np.round(cy, 2),np.round(phi),np.round(rho, 2),np.round(theta)]))
        #ЗДобавление надписи в в кадр
        cv2.putText(frame2, izm[-1], (10, 500), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        # Показ преобразованного изображения после перспективы и его масок
        cv2.imshow('transform', frame2)
        cv2.imshow('mask0', op0)
        cv2.imshow('mask1', op1)
        
    # Наложения слоя с точками границ поля на текущий кадр
    frame = cv2.bitwise_or(frame,img)
    # Показ кадра в окне вывода видео
    cv2.imshow('video', frame)
    # считывание событий отклика мыши мыши
    cv2.setMouseCallback('video',check_points)
    
    # считывание клавишы клавиатуры каждые 30мс
    #Если нажата клавиа <q>, то прервать считывание видеопотока
    key = cv2.waitKey(30)
    if key == ord('q'):
        break

#освобождение памяти переменной cam
cam.release()
#Закрытие всех окон opencv
cv2.destroyAllWindows()


# Список событий OpenCV

In [60]:
events = [i for i in dir(cv2) if 'EVENT' in i]
events

['EVENT_FLAG_ALTKEY',
 'EVENT_FLAG_CTRLKEY',
 'EVENT_FLAG_LBUTTON',
 'EVENT_FLAG_MBUTTON',
 'EVENT_FLAG_RBUTTON',
 'EVENT_FLAG_SHIFTKEY',
 'EVENT_LBUTTONDBLCLK',
 'EVENT_LBUTTONDOWN',
 'EVENT_LBUTTONUP',
 'EVENT_MBUTTONDBLCLK',
 'EVENT_MBUTTONDOWN',
 'EVENT_MBUTTONUP',
 'EVENT_MOUSEHWHEEL',
 'EVENT_MOUSEMOVE',
 'EVENT_MOUSEWHEEL',
 'EVENT_RBUTTONDBLCLK',
 'EVENT_RBUTTONDOWN',
 'EVENT_RBUTTONUP']

# Измерения

In [6]:
izm

['x: 2.71; y: 3.16; phi: 69.0; rho: 4.16; theta: 49.0',
 'x: 2.71; y: 3.16; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.16; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.16; phi: 70.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 70.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 70.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.16; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.16; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 70.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 70.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 68.0; rho: 4.16; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 68.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 69.0; rho: 4.17; theta: 49.0',
 'x: 2.71; y: 3.17; phi: 70.0; rho: 4.17; theta: